## 10分类 10个神经元 单层神经网络

In [1]:
import tensorflow as tf

#### 如果需要使用tf 1.0
tf v2.0 does not have placeholder(), .reset_default_graph().

In [2]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 

Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
#读数据：
import os
import pickle as cPickle
import numpy as np

CIFAR_DIR = "../../../other_datasets/cifar-10-batches-py"
print(os.listdir(CIFAR_DIR))

['data_batch_1', '.DS_Store', 'readme.html', 'batches.meta', 'data_batch_2', 'data_batch_5', 'test_batch', 'data_batch_4', 'data_batch_3']


In [4]:
#将数据读取,返回data 和 labels
def load_data(filename):
    """read data from data file."""
    with open(filename, 'rb') as f:
        data = cPickle.load(f, encoding = 'latin1')
        return data['data'], data['labels']

#### tf 1.0
1. 先把图搭建起来，然后再塞入数据
2. 所以需要占位符，是之后塞入数据的地方

#### X,y的大小
1. 下面的x之所以是[None, 3072], 是因为我们的样本数是不确定的，但是特征量是确定的
2. 样本数不确定：因为是mini batch训练，不确定有多少个

In [5]:
x = tf.placeholder(tf.float32, [None, 3072])
y = tf.placeholder(tf.int64, [None])

### tf.get_variable
1. 获取变量的函数
2. 第一个参数：'w'， 意思是如果没有w这个变量，就定义一个新的。

### W -- 更改
1. w这里定义为一个列向量：有多少个特征，就有多少行（3072行） #这里的x.get_shape()[-1]就是3072
2. w的初始值用正态分布：initializer=tf.random_normal_initializer(0,1)

In [6]:
w = tf.get_variable('w', 
                    [x.get_shape()[-1], 10], 
                    initializer=tf.random_normal_initializer(0,1)) 

### b -- 更改
1. b的维度和w的第二维度一样
2. 本例中，因为w只是一个列向量，所以第二维度是1
3. 或者说，本例中只有一个神经元，所以b也只有1个
4. 初始值一般都是常量，0.0

In [7]:
b = tf.get_variable('b', 
                    [10], 
                    initializer=tf.constant_initializer(0.0)) 

### y -- 更改

[softmax](https://www.tensorflow.org/api_docs/python/tf/nn/softmax)

In [8]:
y_ = tf.matmul(x, w) + b

### 损失函数1：平方差损失函数 mean square loss -- 更改

In [9]:
# p_y_1 = tf.nn.sigmoid(y_)
p_y = tf.nn.softmax(y_) #https://www.tensorflow.org/api_docs/python/tf/nn/softmax

# our course: 1 + e^(wx)
# api: 没有1，直接对e^(wx)做归一化, 即 e^(wx) / sum(e^(wx))

# y_reshaped = tf.reshape(y, (-1, 1))
# y_reshaped_float = tf.cast(y_reshaped, tf.float32)
#loss = tf.reduce_mean(tf.square(y_reshaped_float - p_y_1))

#之前y的shape是[None,] 但是p_y的shape是[None, 10]
#可能的真值y：[0,5,6,3] 一共有10个类
#可能的p_y:[[样本1的十个类别的概率：0.001, 0.9, 0.03, .., 0.01], [样本2的十个类别的概率], ]
#所以我们要把y转化成one hot编码，例如3就是[0,0,0,1,0,0,0,0,0,0]
y_one_hot = tf.one_hot(y, 10, dtype = tf.float32)
loss = tf.reduce_mean(tf.square(y_one_hot - p_y))

### 损失函数：交叉熵函数

In [ ]:
loss2 = tf.losses.sparse_softmax_cross_entropy(labels = y, logits = y_)
#y: 真值
#y_: W*x 内积值
# 这个函数的内部实现了：
# y_ -> softmax
# y -> one_hot
# loss = y*log(y_)

### 形状总结
1. x : (None, 3072)
2. y : (None, )
3. W : (3072, 10)
4. b : (10, )
5. y_ : x * W = (None, 3072) * (3072, 10) = (None, 10)
6. y 和 y_ 的shape不同！

In [10]:
#bool:
#predict = p_y_1 > 0.5

#indexes(int):
#第一个维度是同一类别下每个样本，第二维度是同一个样本下的每个类别（一个类别有一个概率值）
predict_indexes = tf.argmax(y_, 1) #1代表第二个维度，求一个样本，哪个类别的概率值最大

#int64:
#predict_float = tf.cast(predict, tf.int64)

#bool:
correct_prediction = tf.equal(predict_indexes, y) #两个int64的array比较

#float64:
correct_prediction_float = tf.cast(correct_prediction, tf.float32)

#float64, 因为correct_prediction_float是[1.0, 0.0, 1.0, ...] 而reduce_mean就是看有多少1，然后除以len
accuracy = tf.reduce_mean(correct_prediction_float)

### AdamOptimizer
AdamOptimizer(learning_rate)：梯度下降的变种，是反向传播算法的变种，初始learning_rate是0.01，目标最小化loss

In [11]:
with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

### cifar10 数据处理
因为mini-batch，所以需要在cifar10上循环遍历数据，每次在sess.run()的feed_dict塞入不同数据

In [14]:
#filenames: 对 训练数据集 和 测试数据集 是分开的
#need_shuffle: 训练数据集需要shuffle，测试数据集不需要shuffle
class CifarData:
    def __init__(self, filenames, need_shuffle):
        #读入数据：
        all_data = []
        all_labels = []
        for filename in filenames:
            data, labels = load_data(filename)
            #data的对应位置的图像的label，也是label对应位置的label，所以把它们两个zip起来
            all_data.append(data)
            all_labels.append(labels)
            #不再是只调取0，1类 
#             for item, label in zip(data, labels):
#                 if label in [0,1]:
#                     all_data.append(item)
#                     all_labels.append(label)
        #data是numpy的矩阵，
        #item是numpy的向量
        #all_data里面包含了许多numpy的向量，现在我们将这些向量纵向合并到一起，成为一个矩阵：用np.vstack()
        #all_labels里面包含了许多label的向量，现在我们将这些向量横向合并到一起，成为一个矩阵（但是这个矩阵只有一行：n * 1）：用np.hstack()
        self._data = np.vstack(all_data) 
        self._data = self._data / 127.5 - 1 #最后做的改进：将0-255的值二分成0，1
        self._labels = np.hstack(all_labels)
        #print(self._data.shape)
        #print(self._labels.shape)
        self._num_examples = self._data.shape[0] #多少个样本
        self._need_shuffle = need_shuffle
        self._indicator = 0 #遍历到数据集的哪个位置
        if self._need_shuffle:
            self._shuffle_data()
    
    def _shuffle_data(self): #在训练集上shuffle
        # np.random.permutation(6) -> [3,2,5,1,4,0]
        p = np.random.permutation(self._num_examples) #得到一个混的全排
        self._data = self._data[p]
        self._labels = self._labels[p]
        
    def next_batch(self, batch_size): #每次返回batch_size个数个样本
        """return batch_size examples as a batch."""
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data() #就算遍历完，可是如果可以shuffle，我们的数据可以复用
                self._indicator = 0
                end_indicator = batch_size + 0
            else:
                raise Exception("have no more examples") #已经遍历完了并且不能shuffle
        if end_indicator > self._num_examples:
            #此时说明我们输入的batch_size比我们的所有样本都要大
            raise Exception("batch size is larger than example size")
        batch_data = self._data[self._indicator : end_indicator]
        batch_labels = self._labels[self._indicator : end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels

### 测试我们的CifarData
1. 首先cifar10拥有50k个样本（10个类别），我们现在是二分类问题，只选取label为0，1的
2. 所以应该有 50k * (1/5) = 10k个样本 (10000, 3072)

In [15]:
train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' % i) for i in range(1, 6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch') ]

train_data = CifarData(train_filenames, True)

In [16]:
test_data = CifarData(test_filenames, False)

In [17]:
batch_data, batch_labels = train_data.next_batch(3)
# print(batch_data.shape)
# print(batch_data)
# print(batch_labels.shape)
# print(batch_labels)

### 执行流程图

In [18]:
init = tf.global_variables_initializer() #执行初始化
#开启Session，相当于打开绘画
#sess.run()执行计算图
#1. 计算目标:
#表示要训练：[loss, accuracy, train_op]
#表示要测试：[loss, accuracy]

#2. feed_dict:塞入数据
# x: x要塞的数据
# 因为mini-batch，所以需要在cifar10上循环遍历数据，每次塞入不同数据

batch_size = 20 #每次取20张图来 训练 或者 测试
train_steps = 10000
test_steps = 100

with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        #因为train_op返回的东西没有意义，所以用 _ 下划线接住返回值
        loss_val, accu_val, _ = sess.run(
            [loss, accuracy, train_op], 
            feed_dict = {
                x: batch_data, 
                y: batch_labels})  #注意格式！
        #===============================
        #======训练=====================
        if (i+1) % 1000 == 0: #使用(i+1)就可以输出500和1000
            print("[Train] Step: %d, loss: %4.5f, acc: %4.5f"\
                 % (i+1, loss_val, accu_val))
        #===============================
        #======测试=====================
        if (i+1) % 5000 == 0: #使用(i+1)就可以输出500和1000
            test_data = CifarData(test_filenames, False)
            all_test_acc_vals = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels \
                    = test_data.next_batch(batch_size)
                test_acc_val = sess.run(
                    [accuracy],
                    feed_dict = {
                        x: test_batch_data,
                        y: test_batch_labels
                    })
                all_test_acc_vals.append(test_acc_val)
            test_acc = np.mean(all_test_acc_vals)
            print("[Test] Step: %d, acc: %4.5f" % (i+1, test_acc))
            
            


[Train] Step: 1000, loss: 0.14752, acc: 0.25000
[Train] Step: 2000, loss: 0.17704, acc: 0.10000
[Train] Step: 3000, loss: 0.12999, acc: 0.35000
[Train] Step: 4000, loss: 0.11720, acc: 0.40000
[Train] Step: 5000, loss: 0.18303, acc: 0.05000
[Test] Step: 5000, acc: 0.29900
[Train] Step: 6000, loss: 0.16591, acc: 0.15000
[Train] Step: 7000, loss: 0.15722, acc: 0.20000
[Train] Step: 8000, loss: 0.11676, acc: 0.40000
[Train] Step: 9000, loss: 0.13877, acc: 0.30000
[Train] Step: 10000, loss: 0.13881, acc: 0.30000
[Test] Step: 10000, acc: 0.34100


#### 观察
1. train的acc只有0.3左右，但是随机猜的正确率是0.1
2. 上面的test结果，acc增加